# Explore here

In [40]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD

import warnings

In [27]:
df = pd.read_csv('https://breathecode.herokuapp.com/asset/internal-link?id=2326&path=adult-census-income.csv')
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [28]:
df.shape

(32561, 15)

In [29]:
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [30]:
duplicados = df.duplicated()
num_duplicados = duplicados.sum()
print(num_duplicados)

24


Dado que hay un número de duplicados muy bajo respecto del total de datos, estos van a ser eliminados del dataset.

In [31]:
df = df.drop_duplicates(keep='first')
df.shape

(32537, 15)

In [32]:
#Tratar valors faltantes
df = df.replace("?", np.nan)
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [33]:
#Valores faltantes
df.isnull().sum().sort_values(ascending=False)

occupation        1843
workclass         1836
native.country     582
fnlwgt               0
education            0
education.num        0
age                  0
marital.status       0
relationship         0
sex                  0
race                 0
capital.gain         0
capital.loss         0
hours.per.week       0
income               0
dtype: int64

Dado que tenemos tres variables categóricas con valores faltantes, se las va a tratar con una imputación para no intentar perder la mínima información.

In [34]:
df_clean = df.copy()

# Contar valores antes
workclass_antes = df_clean['workclass'].isnull().sum()
occupation_antes = df_clean['occupation'].isnull().sum()
country_antes = df_clean['native.country'].isnull().sum()

# Imputar todos a la vez sin warnings
df_clean = df_clean.fillna({
    'workclass': 'Unknown',
    'occupation': 'Unknown', 
    'native.country': df_clean['native.country'].mode()[0]
})

print(f"✅ Valores imputados:")
print(f"   - workclass: {workclass_antes} → 'Unknown'")
print(f"   - occupation: {occupation_antes} → 'Unknown'") 
print(f"   - native.country: {country_antes} → '{df_clean['native.country'].mode()[0]}'")
print(f"📊 Filas preservadas: {len(df_clean):,} (100% de los datos)")

✅ Valores imputados:
   - workclass: 1836 → 'Unknown'
   - occupation: 1843 → 'Unknown'
   - native.country: 582 → 'United-States'
📊 Filas preservadas: 32,537 (100% de los datos)


#### Codificación de variables categóricas

In [35]:
from IPython.display import display

categorical_cols = df_clean.select_dtypes(include=["object"]).columns 
print("=== ANÁLISIS DETALLADO CATEGÓRICAS ===\n")
    
for col in categorical_cols:
    print(f"📊 {col.upper()}:")
    
    # Trabajar con copia de los datos
    col_data = df_clean[col].copy()
    value_counts = col_data.value_counts()
    n_unique = len(value_counts)  
    
    print(f"   Total valores únicos: {n_unique}")

    display(value_counts) 
      
    if n_unique > 20:
        print(f"   ⚠️  ALTA CARDINALIDAD - considerar agrupación")
    elif n_unique > 10:
        print(f"   💡 Cardinalidad moderada")
    else:
        print(f"   ✅ Cardinalidad baja - ideal para One-Hot Encoding")
        
    print("-" * 50)

=== ANÁLISIS DETALLADO CATEGÓRICAS ===

📊 WORKCLASS:
   Total valores únicos: 9


workclass
Private             22673
Self-emp-not-inc     2540
Local-gov            2093
Unknown              1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: count, dtype: int64

   ✅ Cardinalidad baja - ideal para One-Hot Encoding
--------------------------------------------------
📊 EDUCATION:
   Total valores únicos: 16


education
HS-grad         10494
Some-college     7282
Bachelors        5353
Masters          1722
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           645
Prof-school       576
9th               514
12th              433
Doctorate         413
5th-6th           332
1st-4th           166
Preschool          50
Name: count, dtype: int64

   💡 Cardinalidad moderada
--------------------------------------------------
📊 MARITAL.STATUS:
   Total valores únicos: 7


...

   ✅ Cardinalidad baja - ideal para One-Hot Encoding
--------------------------------------------------
📊 NATIVE.COUNTRY:
   Total valores únicos: 41


native.country
United-States                 29735
Mexico                          639
Philippines                     198
Germany                         137
Canada                          121
Puerto-Rico                     114
El-Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican-Republic               70
Vietnam                          67
Guatemala                        62
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
Greece                           29
France       

   ⚠️  ALTA CARDINALIDAD - considerar agrupación
--------------------------------------------------
📊 INCOME:
   Total valores únicos: 2


income
<=50K    24698
>50K      7839
Name: count, dtype: int64

   ✅ Cardinalidad baja - ideal para One-Hot Encoding
--------------------------------------------------


In [36]:
# Variables de baja cardinalidad - OneHotEncoder
baja_cardinalidad = ['workclass', 'marital.status', 'relationship', 'race', 'sex']
baja_cardinalidad = [col for col in baja_cardinalidad if col in df_clean.columns]

encoder_baja = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoded_baja = encoder_baja.fit_transform(df_clean[baja_cardinalidad])

feature_names_baja = []
for i, col in enumerate(baja_cardinalidad):
    for category in encoder_baja.categories_[i][1:]:
        feature_names_baja.append(f"{col}_{category}")

df_encoded = pd.DataFrame(encoded_baja, columns=feature_names_baja, index=df_clean.index)

In [37]:
# Variables de cardinalidad moderada - OneHotEncoder
moderada_cardinalidad = ['education', 'occupation']
encoder_moderada = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoded_moderada = encoder_moderada.fit_transform(df_clean[moderada_cardinalidad])

feature_names_moderada = []
for i, col in enumerate(moderada_cardinalidad):
    for category in encoder_moderada.categories_[i][1:]:
        feature_names_moderada.append(f"{col}_{category}")

df_moderada = pd.DataFrame(encoded_moderada, columns=feature_names_moderada, index=df_clean.index)
df_encoded = pd.concat([df_encoded, df_moderada], axis=1)

In [38]:
# native.country - Agrupar y luego OneHotEncoder
top_countries = df_clean['native.country'].value_counts().head(10).index
df_country_processed = df_clean['native.country'].apply(lambda x: x if x in top_countries else 'Other')

encoder_country = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
encoded_country = encoder_country.fit_transform(df_country_processed.values.reshape(-1, 1))

feature_names_country = []
for category in encoder_country.categories_[0][1:]:
    feature_names_country.append(f"country_{category}")

df_country = pd.DataFrame(encoded_country, columns=feature_names_country, index=df_clean.index)
df_encoded = pd.concat([df_encoded, df_country], axis=1)

In [39]:
# Añadir variables numéricas
numerical_cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']
for col in numerical_cols:
    df_encoded[col] = df_clean[col]

In [43]:
# Variable objetivo - LabelEncoder
df_encoded['income_encoded'] = LabelEncoder().fit_transform(df_clean['income'])

print(f"Dataset final: {df_encoded.shape}")

Dataset final: (32537, 70)


#### Escalado

In [44]:
# Split
X = df_encoded.drop('income_encoded', axis=1)
y = df_encoded['income_encoded']

# Columnas numéricas que necesitan escalado
numeric_cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']
numeric_cols = [col for col in numeric_cols if col in X.columns]

# Las columnas One-Hot (0/1) NO se escalan
one_hot_cols = [col for col in X.columns if col not in numeric_cols]

print(f"🔢 Numéricas a escalar: {len(numeric_cols)}")
print(f"🔤 One-Hot a mantener: {len(one_hot_cols)}")

🔢 Numéricas a escalar: 6
🔤 One-Hot a mantener: 63


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [46]:
# Escalar solo las numéricas
scaler = StandardScaler()

# Crear copias
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Aplicar escalado solo a columnas numéricas
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

Ahora que el preprocesamiento de los datos ya está hecho